In [1]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
artist_matrix = np.load('artist_matrix2.npy').item()

In [3]:
user_matrix = np.load('user_matrix1.npy').item()

In [4]:
import csv
import random
train_file = 'train.csv'
train_data = {}
test_data = {}

with open(train_file, 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    next(train_csv, None)
    for row in train_csv:

        user   = row[0]
        artist = row[1]
        plays  = int(row[2])
        
        if random.random() < 0.01:
            if not user in test_data:
                test_data[user] = {}

            test_data[user][artist] = plays
        else:

            if not user in train_data:
                train_data[user] = {}

            train_data[user][artist] = plays

print len(train_data.keys())

233286


In [5]:
plays_array  = []
user_medians = {}
for user, user_data in train_data.iteritems():
    user_plays = []
    for artist, plays in user_data.iteritems():
        plays_array.append(plays)
        user_plays.append(plays)

    user_medians[user] = np.median(np.array(user_plays))
global_median = np.median(np.array(plays_array))

In [6]:
data = list()

for user, user_data in train_data.iteritems():
    for artist, plays in user_data.iteritems():
        row = artist_matrix[artist] + user_matrix[user] + [user_medians[user]] + [plays]
        data.append(row)

In [7]:
tdata = list()

for user, user_data in test_data.iteritems():
    for artist, plays in user_data.iteritems():
        row = artist_matrix[artist] + user_matrix[user] + [user_medians[user]] + [plays]
        tdata.append(row)

In [8]:
a = np.array(data)

In [9]:
b = np.array(tdata)

In [10]:
X = a[:,:-1]
Y = a[:,-1]

In [11]:
tX = b[:,:-1]
tY = b[:,-1]

In [12]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=0, n_estimators=10,n_jobs=-1)
rf.fit(X,Y)



RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=-1, oob_score=False, random_state=0,
           verbose=0, warm_start=False)

In [14]:
from sklearn.metrics import mean_absolute_error
y_pred = rf.predict(tX)
mean_absolute_error(tY, y_pred)

172.31587973313913